# Cross-View Visualization Tool

Author: Ted de Vries Lentsch

First version on February 24, 2022

Last update on March 9, 2022

## Import necessary libraries

In [ ]:
# standard libraries
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

# widgets
import ipywidgets

## VIGOR dataset

## 1. Explore dataset

### 1.1. Load data
Below, the names of the images are obtained for the indicated city. The VIGOR dataset is in the same folder as this notebook.

In [ ]:
# VIGOR dataset info
vigor_dir            = "VIGOR"
cities               = ["Chicago","NewYork","SanFrancisco","Seattle"]
streetfolder_name    = "panorama"
satellitefolder_name = "satellite"
combinationinfo_name = "splits"

city_name = cities[3] # CHANGE THIS TO SHOW OTHER CITY

In [ ]:
# paths
root_dir            = os.getcwd()
dataset_dir         = os.path.join(root_dir, vigor_dir)
streetfolder_dir    = os.path.join(dataset_dir, city_name, streetfolder_name)
satellitefolder_dir = os.path.join(dataset_dir, city_name, satellitefolder_name)
combination_dir     = os.path.join(dataset_dir, combinationinfo_name, city_name, "pano_label_balanced.txt")

In [ ]:
# get image names
street_img_names    = list(sorted(os.listdir(streetfolder_dir)))
satellite_img_names = list(sorted(os.listdir(satellitefolder_dir)))

print('The city has {} street images and {} satellite images!'.format(len(street_img_names), len(satellite_img_names)))

### 1.2. Show images
Below, the images of the VIGOR dataset are displayed with an interactive widget. The number of image pairs that can be shown with the widget has been set to 100 because the notebook freezes when this number is too high (like 20k). The semi-positive satellite images are also plotted.

In [ ]:
def get_satellite_and_deltas(street_img_name, combination_dir):
    data_list = []
    with open(combination_dir, 'r') as file:
        for line in file.readlines():
            data = line.split(' ')
            if data[0]==street_img_name:
                for idx in range(4):
                    data_list.append((data[3*idx+1], float(data[3*idx+2]), float(data[3*idx+3])))
                break
    return data_list

In [ ]:
def plot_img(streetfolder_dir, satellitefolder_dir, combination_dir, street_img_name):  
    # load images
    data_list = get_satellite_and_deltas(street_img_name, combination_dir)
    street_img = cv2.imread(os.path.join(streetfolder_dir, street_img_name))[:,:,::-1]
    satellite_img1 = cv2.imread(os.path.join(satellitefolder_dir, data_list[0][0]))[:,:,::-1]
    satellite_img2 = cv2.imread(os.path.join(satellitefolder_dir, data_list[1][0]))[:,:,::-1]
    satellite_img3 = cv2.imread(os.path.join(satellitefolder_dir, data_list[2][0]))[:,:,::-1]
    satellite_img4 = cv2.imread(os.path.join(satellitefolder_dir, data_list[3][0]))[:,:,::-1]    
    W, H, A = street_img.shape[1], street_img.shape[0], satellite_img1.shape[0]
    
    # get deltas
    delta1 = data_list[0][1:]
    delta2 = data_list[1][1:]
    delta3 = data_list[2][1:]
    delta4 = data_list[3][1:]
    
    # create plot
    fig = plt.figure(figsize=[15, 11])
    grid = plt.GridSpec(2, 3, wspace=0.1, hspace=0.05)
    ax1 = plt.subplot(grid[0, :2])
    ax2 = plt.subplot(grid[0, 2])
    ax3 = plt.subplot(grid[1, 0])
    ax4 = plt.subplot(grid[1, 1])
    ax5 = plt.subplot(grid[1, 2])
    
    # plot street view
    ax1.imshow(street_img, extent=(0, W, H, 0), zorder=-10)
    ax1.set_title('Street View', pad=10, fontsize=24)
    ax1.set_xlim(0, W)
    ax1.set_ylim(H, 0)  

    # plot satellite view (positive)
    ax2.imshow(satellite_img1, extent=(0, A, A, 0), zorder=-10)
    ax2.set_title('Satellite View (Positive)', pad=16, fontsize=24)
    ax2.set_xlim(0, A)
    ax2.set_ylim(A, 0)
    
    # plot satellite view (semi-positive 1)
    ax3.imshow(satellite_img2, extent=(0, A, A, 0), zorder=-10)
    ax3.set_title('Semi-Positive 1', pad=10, fontsize=24)
    ax3.set_xlim(0, A)
    ax3.set_ylim(A, 0)

    # plot satellite view (semi-positive 2)
    ax4.imshow(satellite_img3, extent=(0, A, A, 0), zorder=-10)
    ax4.set_title('Semi-Positive 2', pad=10, fontsize=24)
    ax4.set_xlim(0, A)
    ax4.set_ylim(A, 0)

    # plot satellite view (semi-positive 3)
    ax5.imshow(satellite_img4, extent=(0, A, A, 0), zorder=-10)
    ax5.set_title('Semi-Positive 3', pad=10, fontsize=24)
    ax5.set_xlim(0, A)
    ax5.set_ylim(A, 0)
    
    # plot rays
    colors = ['springgreen', 'deepskyblue', 'orange', 'magenta'] # North, East, South, West 
    xc1, yc1 = A/2-delta1[1], A/2+delta1[0] # see GitHub of VIGOR for this formula
    xc2, yc2 = A/2-delta2[1], A/2+delta2[0] # see GitHub of VIGOR for this formula
    xc3, yc3 = A/2-delta3[1], A/2+delta3[0] # see GitHub of VIGOR for this formula
    xc4, yc4 = A/2-delta4[1], A/2+delta4[0] # see GitHub of VIGOR for this formula
    
    ax1.vlines(x=0.00*W, ymin=0, ymax=H, color=colors[2], linewidth=3, zorder=10) # South
    ax1.vlines(x=0.25*W, ymin=0, ymax=H, color=colors[3], linewidth=3, zorder=10) # West
    ax1.vlines(x=0.50*W, ymin=0, ymax=H, color=colors[0], linewidth=3, zorder=10) # North
    ax1.vlines(x=0.75*W, ymin=0, ymax=H, color=colors[1], linewidth=3, zorder=10) # East
    ax1.vlines(x=1.00*W, ymin=0, ymax=H, color=colors[2], linewidth=3, zorder=10) # South

    ax2.scatter(xc1, yc1, s=150, color="yellow", zorder=20) # Center
    ax2.vlines(x=xc1, ymin=0, ymax=yc1, color=colors[0], linewidth=3, zorder=10) # North
    ax2.hlines(y=yc1, xmin=xc1, xmax=A, color=colors[1], linewidth=3, zorder=10) # East
    ax2.vlines(x=xc1, ymin=yc1, ymax=A, color=colors[2], linewidth=3, zorder=10) # South
    ax2.hlines(y=yc1, xmin=0, xmax=xc1, color=colors[3], linewidth=3, zorder=10) # West

    ax3.scatter(xc2, yc2, s=150, color="yellow", zorder=20) # Center
    ax3.vlines(x=xc2, ymin=0, ymax=yc2, color=colors[0], linewidth=3, zorder=10) # North
    ax3.hlines(y=yc2, xmin=xc2, xmax=A, color=colors[1], linewidth=3, zorder=10) # East
    ax3.vlines(x=xc2, ymin=yc2, ymax=A, color=colors[2], linewidth=3, zorder=10) # South
    ax3.hlines(y=yc2, xmin=0, xmax=xc2, color=colors[3], linewidth=3, zorder=10) # West

    ax4.scatter(xc3, yc3, s=150, color="yellow", zorder=20) # Center
    ax4.vlines(x=xc3, ymin=0, ymax=yc3, color=colors[0], linewidth=3, zorder=10) # North
    ax4.hlines(y=yc3, xmin=xc3, xmax=A, color=colors[1], linewidth=3, zorder=10) # East
    ax4.vlines(x=xc3, ymin=yc3, ymax=A, color=colors[2], linewidth=3, zorder=10) # South
    ax4.hlines(y=yc3, xmin=0, xmax=xc3, color=colors[3], linewidth=3, zorder=10) # West

    ax5.scatter(xc4, yc4, s=150, color="yellow", zorder=20) # Center
    ax5.vlines(x=xc4, ymin=0, ymax=yc4, color=colors[0], linewidth=3, zorder=10) # North
    ax5.hlines(y=yc4, xmin=xc4, xmax=A, color=colors[1], linewidth=3, zorder=10) # East
    ax5.vlines(x=xc4, ymin=yc4, ymax=A, color=colors[2], linewidth=3, zorder=10) # South
    ax5.hlines(y=yc4, xmin=0, xmax=xc4, color=colors[3], linewidth=3, zorder=10) # West    
    
    # disable ticks
    ax1.axis('off')
    ax2.axis('off')
    ax3.axis('off')
    ax4.axis('off')
    ax5.axis('off')

    plt.show()

    # print image names
    print("Street view:                      {}".format(street_img_name))
    print("Satellite view (positive):        {}".format(data_list[0][0]))
    print("Satellite view (semi-positive 1): {}".format(data_list[1][0]))
    print("Satellite view (semi-positive 2): {}".format(data_list[2][0]))
    print("Satellite view (semi-positive 3): {}".format(data_list[3][0]))

In [ ]:
ipywidgets.interact(lambda idx: plot_img(streetfolder_dir=streetfolder_dir,
                                         satellitefolder_dir=satellitefolder_dir,
                                         combination_dir=combination_dir,
                                         street_img_name=street_img_names[idx]),
                                idx=range(min(len(street_img_names), 100)))